In [19]:
import os

In [20]:
import torch

In [21]:
import torch.nn as nn

In [22]:
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kullanılan aygıt:", device)

Kullanılan aygıt: cpu


In [24]:
DATA_DIR = r"C:\Users\Şebnem\Desktop\animals\raw-img"


In [25]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

In [26]:
full_dataset = datasets.ImageFolder(DATA_DIR, transform=transform)
class_names = full_dataset.classes
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32)

In [27]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 32 * 32, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [28]:
model = SimpleCNN(num_classes=len(class_names)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [29]:
EPOCHS = 6
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    acc = 100 * correct / total
    print(f"📚 Epoch {epoch+1}/{EPOCHS} - Loss: {running_loss:.4f} - Accuracy: {acc:.2f}%")


100%|████████████████████████████████████████████████████████████████████████████████| 655/655 [16:23<00:00,  1.50s/it]


📚 Epoch 1/6 - Loss: 1193.2433 - Accuracy: 36.04%


100%|████████████████████████████████████████████████████████████████████████████████| 655/655 [19:37<00:00,  1.80s/it]


📚 Epoch 2/6 - Loss: 852.9288 - Accuracy: 55.63%


100%|████████████████████████████████████████████████████████████████████████████████| 655/655 [27:18<00:00,  2.50s/it]


...

📚 Epoch 4/6 - Loss: 401.9341 - Accuracy: 79.55%


100%|████████████████████████████████████████████████████████████████████████████████| 655/655 [18:23<00:00,  1.69s/it]


📚 Epoch 5/6 - Loss: 195.4417 - Accuracy: 89.99%


100%|████████████████████████████████████████████████████████████████████████████████| 655/655 [18:42<00:00,  1.71s/it]

📚 Epoch 6/6 - Loss: 79.2649 - Accuracy: 96.33%


In [30]:
torch.save(model.state_dict(), "simple_cnn_animals.pth")
print("Model başarıyla kaydedildi.")

Model başarıyla kaydedildi.
